In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

### Pruebo con np.histogram

In [9]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

### Pruebo con Quantile

In [25]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(copy=False)

In [33]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 0.86886887  0.86636637]
True
True


### Pruebo con RobustScaler

In [37]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(copy=False)

In [38]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 1.1875      0.97560976]
True
True


### Busco hiper-parametros con random search. Mejor resultado: np.histogram

In [39]:
%%notify

rf = RandomForestClassifier(n_jobs=-1)

bootstrap=[False]
max_features =["auto","sqrt","log2",None]

param_grid = {"bootstrap": bootstrap,"max_features":max_features}
custom_cv = ShuffleSplit(n_splits=10, test_size=0.2)

grid_search = GridSearchCV(rf, param_grid=param_grid ,cv=custom_cv)
start = time()
grid_search.fit(X, y)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(grid_search.cv_results_['params'])))
print("")
score.report_single(grid_search.cv_results_)

GridSearchCV duro 87.30 segundos para 4 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.583 (std: 0.004)
Promedio validation score: 0.248 (std: 0.012)
Promedio score time: 0.303s
Hiper-parametros: {'max_features': 'sqrt', 'bootstrap': False}

Puesto: 2
Promedio training score: 0.583 (std: 0.004)
Promedio validation score: 0.248 (std: 0.012)
Promedio score time: 0.297s
Hiper-parametros: {'max_features': 'auto', 'bootstrap': False}

Puesto: 3
Promedio training score: 0.583 (std: 0.004)
Promedio validation score: 0.247 (std: 0.012)
Promedio score time: 0.317s
Hiper-parametros: {'max_features': 'log2', 'bootstrap': False}

Puesto: 4
Promedio training score: 0.583 (std: 0.004)
Promedio validation score: 0.247 (std: 0.012)
Promedio score time: 0.296s
Hiper-parametros: {'max_features': None, 'bootstrap': False}



<IPython.core.display.Javascript object>